# Mitigating Unfairness in the Law School Dataset

In this example, we will examine the well known Law School Admissions dataset, provided by [Project SEAPHE](http://www.seaphe.org/databases.php). The motivation was to gain a better understanding of race in law school admissions, and ensuring that students who would ultimately pass the bar exam were treated fairly.

## Obtaining the Data

We obtain the data from the `tempeh` package. The main feature data (which we will refer to as $X$) has two features - undergraduate GPA and LSAT score. The label (which we call $y$) is 0 or 1 dependent on whether that student passed the bar exam. Finally, we also have the race of the students ('black' or white') as a sensitive attribute, which we will refer to as $A$.

We start by loading the data, which have already been split into "train" and "test" subsets for us. However, we do need to rescale the two features in $X$ to lie in the range $[0, 1]$:

In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

from tempeh.configurations import datasets
dataset = datasets['lawschool_passbar']()

scaler = MinMaxScaler()

X_train = pd.DataFrame(scaler.fit_transform(dataset.X_train), columns=dataset.features)
X_test = pd.DataFrame(scaler.fit_transform(dataset.X_test), columns=dataset.features)

y_train = pd.Series(dataset.y_train.squeeze(), name="Pass Bar", dtype=int)
y_test = pd.Series(dataset.y_test.squeeze(), name="Pass Bar", dtype=int)

A_train = pd.Series(dataset.race_train, name="Race")
A_test = pd.Series(dataset.race_test, name="Race")

Now, let us examine the data. First, we can look at the breakdown of students by race in the dataset. We see that there are far more white students than black, which is already a suggestion of bias in the data:

In [ ]:
l, c = np.unique(dataset.race_train, return_counts=True)
for i in range(len(l)):
    print("Number of {0} students is {1}".format(l[i], c[i]))

We can also start using the group metrics from `fairlearn` to examine things such as the final pass rate for the bar exam. Both rates are high, although higher for whites:

In [ ]:
from fairlearn.metrics import group_mean_prediction

def group_metric_printer(name, group_metric_result):
    print("{0} overall {1:.3f}".format(name, group_metric_result.overall))
    for k, v in group_metric_result.by_group.items():
        print("{0} for {1:8} {2:.3f}".format(name, k, v))

unused = np.ones(len(dataset.y_train))
group_metric_printer("Pass Rate", group_mean_prediction(unused, y_train, A_train))

Looking at the raw numbers, we see how dominant whites who pass the bar exam are in the dataset:

In [ ]:
for r in ['black', 'white']:
    Ys = y_train[A_train==r]
    print(r)
    print(Ys.value_counts())

We can also examine the [ROC-AUC scores](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html) for each feature (that is LSAT score and undergraduate GPA) for the overall dataset and by race. Used in this way, the ROC-AUC score is a measure of how predictive each feature is of the final label. A score of 0.5 would mean that the feature is no better than a coin flip (a coin biased to produce a desired fraction of positives), while a score of 1 means that the feature is perfectly discriminating:

In [ ]:
from fairlearn.metrics import group_roc_auc_score

for column_name in X_train:
    column_data = X_train[column_name]
    title = "ROC-AUC {0}".format(column_name)
    group_metric_printer(title, group_roc_auc_score(y_train, column_data, A_train))

We can also examine the CDFs for the LSAT and GPAs for whites and blacks (recall that we rescaled both to $[0,1]$ above):

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import cumfreq

def plot_separated_cdf(data, A):
    for a in np.unique(A):
        subset = data[A==a]
        
        cdf = cumfreq(subset, numbins=20)
        x = cdf.lowerlimit + np.linspace(0, cdf.binsize*cdf.cumcount.size, cdf.cumcount.size)
        plt.plot(x, cdf.cumcount / len(subset), label=a)
    plt.xlabel(data.name)
    plt.ylabel("Cumulative Frequency")
    plt.legend()
    plt.show()
        

plot_separated_cdf(X_train['lsat'], A_train)
plot_separated_cdf(X_train['ugpa'], A_train)

## An Unmitigated Predictor

As a point of comparison for later, we can train a predictor without regard to fairness.

In [ ]:
from sklearn.linear_model import LogisticRegression

unmitigated_predictor = LogisticRegression(solver='liblinear', fit_intercept=True)

unmitigated_predictor.fit(X_train, y_train)

With this predictor, we can look at some statistics. First, we can look at the average predictions. Immediately we see that a 100% pass rate for whites is predicted:

In [ ]:
unmitigated_mean_predictions = group_mean_prediction(y_test, # Actually unused
                                                     unmitigated_predictor.predict(X_test),
                                                     A_test)
group_metric_printer("Predicted Pass Rate", unmitigated_mean_predictions)

This suggests that we need to think a little more deeply about what we're doing. Ultimately, we would want to use this model for admissions, and we would want to admit students (fairly) according to their chances of passing the bar exam. The `LogisticRegression` estimator provides a `predict_proba` method for this purpose - this provides the probability of predicting a given class, which is then thresholded by the `predict` method itself.

First, we can obtain the appropriate probabilities:

In [ ]:
y_pred_unmitigated = pd.Series(unmitigated_predictor.predict_proba(X_test)[:,1], name="Pass Probability Unmitigated")

We can now look at the mean predicted probabilities:

In [ ]:
group_metric_printer("Predicted Pass Probability", group_mean_prediction(y_test, y_pred_unmitigated, A_test))
plot_separated_cdf(y_pred_unmitigated, A_test)

We can also look at the ROC-AUC scores for the predictions

In [ ]:
group_roc_auc_score_unmitigated = group_roc_auc_score(y_test, y_pred_unmitigated, A_test)
group_metric_printer("Unmitigated ROC-AUC score", group_roc_auc_score_unmitigated)

## Unfairness Mitigation with Grid Search

In this section, we will attempt to mitigate the unfairness in the incoming data using the `GridSearch` algorithm of `fairlearn`. We shall apply constraints of demographic parity - that is, we will attempt to equalise the positive prediction rates between whites and blacks. This is appropriate for affirmative action scenarios.

We do a grid search in two stages. In the first, we do a low resolution search, identifying a region of promising looking Lagrange multipliers. We will then perform a higher resolution search around that region.

Due to the extreme imbalance in the data, we can't use the default grid for the initial sweep, but must specify one ourselves. We will use the following subroutine for grid generation, which transforms an array of multiplier values into the form required by `GridSearch`:

In [ ]:
def grid_generator(multipliers):
    iterables = [['+','-'], ['all'], ['black', 'white']]
    midx = pd.MultiIndex.from_product(iterables, names=['sign', 'event', 'group_id'])
    
    sweep_lambdas = []
    for l in multipliers:
        nxt = pd.Series(np.zeros(4), index=midx)
        if l < 0:
            nxt[("-", "all", "white")] = abs(l)
        else:
            nxt[("+", "all", "white")] = l
        sweep_lambdas.append(nxt)
            
    return pd.concat(sweep_lambdas, axis=1)

First, the low resolution sweep:

In [ ]:
from fairlearn.reductions import GridSearch, DemographicParity

n_first_sweep = 21
first_multipliers = np.linspace(-10, 10, n_first_sweep)

sweep = GridSearch(LogisticRegression(solver='liblinear', fit_intercept=True),
                   constraints=DemographicParity(),
                   grid=grid_generator(first_multipliers))

sweep.fit(X_train, y_train, sensitive_features=A_train)

We can plot the mean opportunity of these models as a function of the multiplier used. We can also see that the opportunity is equalised for blacks and whites with a multiplier of around six.

In [ ]:
def metric_sweep_plot(multipliers, all_results, metric_func):
    assert len(multipliers)==len(all_results)
    metrics = [metric_func(y_test, x.predictor.predict_proba(X_test)[:,1], A_test)
               for x in all_results]
    
    for r in ['black', 'white']:
        plt.scatter(multipliers, [x.by_group[r] for x in metrics], label=r)
    plt.scatter(multipliers, [x.overall for x in metrics], label='overall')
    plt.xlabel("Multiplier")
    plt.ylabel(metric_func.__name__[6:])
    plt.legend()
    plt.show()
    
metric_sweep_plot(first_multipliers, sweep.all_results, metric_func=group_mean_prediction)

We can examine the ROC-AUC score for this set of models:

In [ ]:
metric_sweep_plot(first_multipliers, sweep.all_results, metric_func=group_roc_auc_score)

Finally, we compare the disparity in the opportunity with the ROC-AUC score for each model. This will give a better idea of the tradeoffs available to us. This shows that we can substantially cut the disparity in opportunity with a minimal effect on the ROC-AUC score. In the second sweep, we shall seek to do this.

In [ ]:
def auc_roc_disparity_sweep_plot(multipliers, all_results):
    assert len(multipliers)==len(all_results)
    
    roc_auc = np.zeros(len(multipliers))
    disparity = np.zeros(len(multipliers))
    
    for i in range(len(multipliers)):
        preds = all_results[i].predictor.predict_proba(X_test)[:,1]
        roc_auc[i] = group_roc_auc_score(y_test, preds, A_test).minimum
        disparity[i] = group_mean_prediction(y_test, preds, A_test).range
        
    plt.scatter(roc_auc, disparity)
    plt.xlabel("Minimum ROC AUC score")
    plt.ylabel("Disparity in Opportunity")
    plt.show()
    
auc_roc_disparity_sweep_plot(first_multipliers, sweep.all_results)

We know that we need $\lambda \approx 6$, so expand a grid around there

In [ ]:
second_multipliers = np.linspace(5, 7, 21)

second_sweep = GridSearch(LogisticRegression(solver='liblinear', fit_intercept=True),
                   constraints=DemographicParity(),
                   grid=grid_generator(second_multipliers))

second_sweep.fit(X_train, y_train, sensitive_features=A_train)

Now we can look at the opportunity as a function of $\lambda$ and the tradeoff between the ROC-AUC score and disparity:

In [ ]:
opportunity_sweep_plot(second_multipliers, second_sweep.all_results)
auc_roc_disparity_sweep_plot(second_multipliers, second_sweep.all_results)

### Analysing Grid Search results

We have used the `GridSearch` algorithm with demographic parity constraints. However, as noted above, for analysing our results, it is more appropriate to look at the predicted probabilities and the ROC-AUC scores.

We can plot these scores against the mean disparity (between blacks and whites) for each model. We can see that for very little change in the worst ROC-AUC score, we can substantially reduce the disparity, as measured by the mean prediction:

In [ ]:
sweep_roc_auc_score = np.zeros(n_second_sweep)
sweep_mean_disparity = np.zeros(n_second_sweep)

for i in range(n_second_sweep):
    preds = second_sweep.all_results[i].predictor.predict_proba(X_test)[:,1]
    sweep_roc_auc_score[i] = group_roc_auc_score(y_test, preds, A_test).minimum
    sweep_mean_disparity[i] = group_mean_prediction(y_test, preds, A_test).range
    
plt.scatter(sweep_roc_auc_score, sweep_mean_disparity)
plt.xlabel("Minimum ROC AUC Score")
plt.ylabel("Disparity")
plt.show()

We can also look at how the predictions are varying with the set of generated Lagrange multipliers. What we see is that we are gradually moving to predict that all students pass the bar. The 'opportunity gap' between the two sets of points matches the range in disparities in the above plot.

In [ ]:
mean_predictions = [group_mean_prediction(y_test, x.predictor.predict_proba(X_test)[:,1], A_test)
                   for x in second_sweep.all_results]

for r in ['black', 'white']:
    plt.scatter(second_sweep_multipliers, [x.by_group[r] for x in mean_predictions], label=r)
plt.xlabel("Multiplier")
plt.ylabel("Opportunity")
plt.legend()
plt.show()

In [ ]:
i_mult = 30
print(second_sweep_multipliers[i_mult])
gs_preds = pd.Series(second_sweep.all_results[i_mult].predictor.predict_proba(X_test)[:,1], name="Predict_Proba")
group_metric_printer("ROC-AUC", group_roc_auc_score(y_test, gs_preds, A_test))
print("mean_prediction_disparity", group_mean_prediction(y_test, gs_preds, A_test).range)
plot_separated_cdf(gs_preds, A_test)

## Mitigation with Threshold Optimisation

We can also use the post-processing approach from `fairlearn`.

In [ ]:
from fairlearn.postprocessing import ThresholdOptimizer

class LogisticRegressionAsRegression:
    def __init__(self, logistic_regression_estimator):
        self.logistic_regression_estimator = logistic_regression_estimator
    
    def fit(self, X, y):
        self.logistic_regression_estimator.fit(X, y)
    
    def predict(self, X):
        # use predict_proba to get real values instead of 0/1, select only prob for 1
        scores = self.logistic_regression_estimator.predict_proba(X)[:,1]
        return scores

est = LogisticRegressionAsRegression(LogisticRegression(solver='liblinear', fit_intercept=True))

postprocess_estimator = ThresholdOptimizer(estimator=est,
                                          constraints="demographic_parity")

postprocess_estimator._plot = True
postprocess_estimator.fit(X_train, y_train, sensitive_features=A_train)

In [ ]:
pp_preds = postprocess_estimator.predict(X_test, sensitive_features=A_test)

In [ ]:
pp_mean_predictions = group_mean_prediction(y_test, # Actually unused
                                            pp_preds,
                                            A_test)
group_metric_printer("Predicted Pass Rate", pp_mean_predictions)

In [ ]:
print(np.unique(pp_preds))

In [ ]:
pp_roc_auc_score = group_roc_auc_score(y_test, pp_preds, A_test)

group_metric_printer("PP ROC-AUC", pp_roc_auc_score)

In [ ]:
for r in ['black', 'white']:
    Ys = y_train[A_train==r]
    print(r)
    print(Ys.value_counts())